# Temas: Regresión
<br/><br/>

<center>
    
## Taller de Ciencia de Datos
### Omar Piña Ramírez
### Instituto Nacional de Perinatología
### Departamento de Bioinformática y Análisis Estadísticos
### Investigador en Ciencias Médicas
### delozath@gmail.com
</center>

In [1]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1500px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
.CodeMirror{
    font-size: 15px;
}

.rendered_html table, .rendered_html td, .rendered_html th {
    font-size: 120%;
}
</style>

In [2]:
import numpy   as np
import pandas  as pd
import seaborn as sns

from   matplotlib import pyplot as plt

import ipywidgets as widgets
from   ipywidgets import interact, interact_manual, FloatSlider, Layout

import chart_studio.plotly as py
import plotly.graph_objs   as go
import plotly.express      as px
from   plotly.offline      import iplot, init_notebook_mode

import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode (connected=True)

## Aprendizaje supervisado -> Regresión

### Datos Infarto

In [3]:
PATH = './data/'
file = 'cardiovascular.csv'

data  = pd.read_csv(PATH + file)
data  = data.drop(columns='ind')
data

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
457,214,0.40,5.98,31.72,Absent,64,28.45,0.00,58,0
458,182,4.20,4.41,32.10,Absent,52,28.61,18.72,52,1
459,108,3.00,1.59,15.23,Absent,40,20.09,26.64,55,0
460,118,5.40,11.61,30.79,Absent,64,27.35,23.97,40,0


In [5]:
cols = ['adiposity','obesity','alcohol']

@interact
def scatter_regression(x=cols,groups=['age','famhist','chd']):
    fig = px.scatter(data,x=x,y='obesity', color=groups,width=650, height=650)
    fig.update_traces(marker={'size': 15})
    fig.show()

interactive(children=(Dropdown(description='x', options=('adiposity', 'obesity', 'alcohol'), value='adiposity'…

In [24]:
y = ['sbp','ldl']

@interact
def scatter_regression(x=cols,y=y,groups=['age','famhist','chd']):
    fig = px.scatter(data,x=x,y=y, color=groups,width=650, height=650)
    fig.update_traces(marker={'size': 15})
    fig.show()

interactive(children=(Dropdown(description='x', options=('adiposity', 'obesity', 'alcohol'), value='adiposity'…

### Scrubbing

In [7]:
replace  = {'famhist':{'Present':1,'Absent':0}}
data_reg = data.drop(columns='adiposity')
data_reg = data_reg.replace(replace)
data_reg

,sbp,tobacco,ldl,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,1,49,25.30,97.20,52,1
1,144,0.01,4.41,0,55,28.87,2.06,63,1
2,118,0.08,3.48,1,52,29.14,3.81,46,0
3,170,7.50,6.41,1,51,31.99,24.26,58,1
4,134,13.60,3.50,1,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...
457,214,0.40,5.98,0,64,28.45,0.00,58,0
458,182,4.20,4.41,0,52,28.61,18.72,52,1
459,108,3.00,1.59,0,40,20.09,26.64,55,0
460,118,5.40,11.61,0,64,27.35,23.97,40,0


In [8]:
col_target   = 'obesity'
col_features = data_reg.columns.to_list(); col_features.remove(col_target)
col_features, col_target

(['sbp', 'tobacco', 'ldl', 'famhist', 'typea', 'alcohol', 'age', 'chd'],
 'obesity')

In [9]:
NT    = 0.8
NV    = 1 - NT
N     = data_reg.shape[0]

In [10]:
X     = data_reg[col_features]
y     = data_reg[col_target]

In [20]:
from sklearn.linear_model import LinearRegression
from scipy.stats          import pearsonr
#from sklearn.metrics         import recall_score as sensitivity

k     = 10
model = LinearRegression()

for i in range(10):
    index    = np.arange(N); np.random.shuffle(index)
    tune     = index[:int(NT*N)]
    validate = index[-int(NV*N):]
    
    X_Train, y_Train = X.iloc[tune    ],y.iloc[tune    ].values.ravel()
    X_Test , y_Test  = X.iloc[validate],y.iloc[validate].values.ravel()
    model.fit(X_Train, y_Train)
    y_Estim = model.predict(X_Train)
    r       = pearsonr(y_Train,y_Estim)
    print("R^2: {:.3f}, p-value: {:.3f}".format(r[0],r[1]))

R^2: 0.436, p-value: 0.000
R^2: 0.401, p-value: 0.000
R^2: 0.471, p-value: 0.000
R^2: 0.428, p-value: 0.000
R^2: 0.401, p-value: 0.000
R^2: 0.482, p-value: 0.000
R^2: 0.433, p-value: 0.000
R^2: 0.440, p-value: 0.000
R^2: 0.400, p-value: 0.000
R^2: 0.418, p-value: 0.000


In [27]:
NT    = 0.8
NV    = 1 - NT
N     = data_reg.shape[0]

replace  = {'famhist':{'Present':1,'Absent':0}}
data_reg = data
data_reg = data_reg.replace(replace)

col_target   = 'obesity'
col_features = data_reg.columns.to_list(); col_features.remove(col_target)

X     = data_reg[col_features]
y     = data_reg[col_target]

k     = 10
model = LinearRegression()

for i in range(10):
    index    = np.arange(N); np.random.shuffle(index)
    tune     = index[:int(NT*N)]
    validate = index[-int(NV*N):]
    
    X_Train, y_Train = X.iloc[tune    ],y.iloc[tune    ].values.ravel()
    X_Test , y_Test  = X.iloc[validate],y.iloc[validate].values.ravel()
    model.fit(X_Train, y_Train)
    y_Estim = model.predict(X_Train)
    r       = pearsonr(y_Train,y_Estim)
    print("R^2: {:.3f}, p-value: {:.3f}".format(r[0],r[1]))

R^2: 0.736, p-value: 0.000
R^2: 0.766, p-value: 0.000
R^2: 0.789, p-value: 0.000
R^2: 0.756, p-value: 0.000
R^2: 0.744, p-value: 0.000
R^2: 0.724, p-value: 0.000
R^2: 0.768, p-value: 0.000
R^2: 0.745, p-value: 0.000
R^2: 0.745, p-value: 0.000
R^2: 0.735, p-value: 0.000
